In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import cv2
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, FPHA, LMDB, DATA_DIR

In [ ]:
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/'exp1/multireso_normuvd_data.cfg')
epoch = 200
exp_dir = cfg["exp_dir"]
data_split = 'test'
split_set = cfg[data_split + '_set']

# Visualize Dataloader

In [ ]:
dataset_kwargs = {'split_set': split_set}
cfg['visual'] = 1
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)

In [ ]:
idx = 0 
for i, (img, img_list, uvd_gt, hand_center_uvd) in enumerate(data_loader):
    if i == idx:
        batch_size = uvd_gt.shape[0]
        img = img.cpu().numpy()
        
        uvd_gt = uvd_gt.cpu().numpy()
        uvd_gt = np.squeeze(uvd_gt)
        
        img0 = img_list[0].cpu().numpy()
        img0 = np.swapaxes(img0, 2, 3)
        img0 = np.swapaxes(img0, 1, 3)
        img0 = IMG.scale_img_255(img0)

        img1 = img_list[1].cpu().numpy()
        img1 = np.swapaxes(img0, 2, 3)
        img1 = np.swapaxes(img0, 1, 3)
        img1 = IMG.scale_img_255(img1)

        img2 = img_list[2].cpu().numpy()
        img2 = np.swapaxes(img0, 2, 3)
        img2 = np.swapaxes(img0, 1, 3)
        img2 = IMG.scale_img_255(img2)           
        
        mean_u = hand_center_uvd[0].cpu().numpy()
        mean_v = hand_center_uvd[1].cpu().numpy()
        mean_z = hand_center_uvd[2].cpu().numpy()
        hand_center_uvd = np.asarray([mean_u, mean_v, mean_z]).T
        _, uvd_gt = FPHA.normuvd2xyzuvd_color(uvd_gt, hand_center_uvd)
        break
    i += 1

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        k = np.ravel_multi_index((i, j), (4, 4))
        if k >= len(img0):
            break
        cur_img = img[idx]
        u = uvd_gt[idx]        
        ax[i, j].imshow(cur_img)
        u = IMG.scale_points_WH(u, (FPHA.ORI_WIDTH, FPHA.ORI_HEIGHT), 
                                (cur_img.shape[0], cur_img.shape[1]))
        FPHA.visualize_joints_2d(ax[i, j], u[FPHA.REORDER_IDX], joint_idxs=False)
        idx += 1

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        k = np.ravel_multi_index((i, j), (4, 4))
        if k >= len(img0):
            break
        cur_img = img0[idx]
        u = uvd_gt[idx]        
        ax[i, j].imshow(cur_img)
        idx += 1

# Evaluation

In [ ]:
keys = LMDB.get_keys(os.path.join(DATA_DIR, split_set + "_keys_cache.p"))
xyz_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + '_xyz_gt.lmdb'), 'float32', (21, 3))
uvd_gt = FPHA.xyz2uvd_color(xyz_gt)
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_uvd.txt'.format(epoch, data_split))
pred_uvd = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_xyz = FPHA.uvd2xyz_color(pred_uvd)

In [ ]:
idx = 13392 
print(keys[idx])
cur_pred_uvd = pred_uvd[idx]
cur_uvd_gt = uvd_gt[idx]
img = np.asarray(Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx])))
fig, ax = plt.subplots()
ax.imshow(img)
FPHA.visualize_joints_2d(ax, pred_uvd[idx][FPHA.REORDER_IDX], joint_idxs=False, c='r')
FPHA.visualize_joints_2d(ax, uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False, c='b')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(221)
ax.imshow(img)
FPHA.visualize_joints_2d(ax, cur_pred_uvd[FPHA.REORDER_IDX], joint_idxs=False, c='r')
FPHA.visualize_joints_2d(ax, cur_uvd_gt[FPHA.REORDER_IDX], joint_idxs=False, c='b')

for proj_idx, (proj_1, proj_2) in enumerate([[0, 1], [1, 2], [0, 2]]):
    ax = fig.add_subplot(2, 2, 2 + proj_idx)
    if proj_idx == 0:
        # Invert y axes to align with image in camera projection
        ax.invert_yaxis()
    ax.set_aspect('equal')
    FPHA.visualize_joints_2d(ax,
                        np.stack(
                            [cur_pred_uvd[FPHA.REORDER_IDX][:, proj_1], 
                             cur_pred_uvd[FPHA.REORDER_IDX][:, proj_2]],
                            axis=1),
                        joint_idxs=False, c='r')
    FPHA.visualize_joints_2d(ax,
                        np.stack(
                            [cur_uvd_gt[FPHA.REORDER_IDX][:, proj_1], 
                             cur_uvd_gt[FPHA.REORDER_IDX][:, proj_2]],
                            axis=1),
                        joint_idxs=False, c='b')    

In [ ]:
print('%s UVD mean_l2_error: ' %data_split, FPHA.mean_L2_error(uvd_gt[:len(pred_uvd)], pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, FPHA.mean_L2_error(xyz_gt[:len(pred_xyz)], pred_xyz))
error = []
for i, (pred, uvd) in enumerate(zip(pred_uvd, uvd_gt)):
#     print(i, FPHA.mean_L2_error(uvd, pred))
    error.append(FPHA.mean_L2_error(uvd, pred))
error = np.asarray(error)
min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)
print('Best Pose id:', min_error_idx, 'uvd_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uvd_error:', error[max_error_idx])
for idx in np.argsort(error):
    print(idx)

In [ ]:
pck = FPHA.percentage_frames_within_error_curve(xyz_gt[:len(pred_uvd)], pred_xyz)
pck_str = ''
for p in pck:
    pck_str += str(p) + ', '
print(pck_str)
thresholds = np.arange(0, 85, 5)
print('AUC:', FPHA.calc_auc(pck, thresholds))